In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
!pip install transformers datasets peft torch accelerate trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.3/318.3 kB 6.2 MB/s eta 0:00:0000:01


# This is **BF16 LoRA fine-tuning** (Bfloat16 Low-Rank Adaptation) without 8-bit quantization. 🚀

### setup optimized and correct for running inference and fine-tuning on a single GPU.
### Main reason because of not ENOUGH RESOURCES 

### **Challenges & Solution Approach**  

#### **Challenges with Fine-Tuning Alone:**  
Fine-tuning alone did not yield optimal results due to:  
1. **Unique Drug Prices:** With **2.5 lakh** unique drug prices, fine-tuning struggled to generalize these values correctly.  
2. **Similar Drug Names:** Many drugs have **similar names**, making it difficult for the model to differentiate between them without explicit context.  
3. **Resource & Time Constraints:** Fine-tuning with a large dataset required **significant computational resources and time** to create structured, high-quality prompts covering all variations.  

#### **Solution: Fine-Tuning + RAG (Retrieval-Augmented Generation)**  
To overcome these challenges, I combined **fine-tuning** with **RAG (Retrieval-Augmented Generation):**  
- **Fine-Tuning:** Trained the model on **well-structured examples** to improve **response formatting** and coherence.  
- **RAG:** Used **vector search (FAISS)** to retrieve the **most relevant drug details** from a structured database, ensuring **accurate and dynamic responses**.  

#### **Outcome:**  
- The model now generates **well-structured responses** efficiently.  
- **Drug prices and specific details** are retrieved accurately from the **database**, avoiding misinterpretation.  
- This hybrid approach allows the model to **handle any drug dataset dynamically** without requiring extensive fine-tuning for every possible drug.

### 

In [3]:
import torch

from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments

from datasets import load_dataset

from peft import LoraConfig, get_peft_model

from trl import SFTTrainer

import transformers

In [4]:
import os



# Disable WandB logging  

os.environ["WANDB_DISABLED"] = "true"  # I have turned off wandb 


For this fine-tuning setup, I’m using **LoRA (Low-Rank Adaptation)** to efficiently adapt a **causal language model (CAUSAL_LM)** while keeping memory usage low.  

### **Training Configuration:**  
I’ve enabled **bfloat16 (bf16)** precision to optimize performance. The **learning rate is set to 5e-6**, following a **cosine scheduler**. I’m training for **2 epochs** with a **warmup ratio of 0.2** to stabilize learning. The batch size is **4 per device** for both training and evaluation.  

To manage resources better, I’ve **enabled gradient checkpointing** and **set gradient accumulation steps to 1**. Logging happens **every 20 steps**, and I’m saving checkpoints **every 100 steps**, keeping only the latest one. The model’s output is stored in `"./checkpoint_dir"` with overwriting enabled.  

### **LoRA Configuration:**  
I’m using **rank 16** for LoRA adaptation with **alpha set to 32** and a **dropout of 0.05** to prevent overfitting. Since this is a causal LM, I’ve targeted **key projection layers (`q_proj`, `k_proj`, `v_proj`, `o_proj`)** along with **MLP layers (`gate_proj`, `up_proj`, `down_proj`)**. **Bias is set to "none"**, meaning no additional bias terms are applied.  

This setup keeps most of the **pretrained model frozen** while fine-tuning only selected layers, making the process both **efficient and scalable**. 🚀

In [5]:
training_config = {

    "bf16": True,

    "do_eval": False,

    "learning_rate": 5.0e-06,

    "log_level": "info",

    "logging_steps": 20,

    "logging_strategy": "steps",

    "lr_scheduler_type": "cosine",

    "num_train_epochs": 2,

    "max_steps": -1,

    "output_dir": "./checkpoint_dir",  # Save model locally

    "overwrite_output_dir": True,

    "per_device_eval_batch_size": 4,

    "per_device_train_batch_size": 4,

    "remove_unused_columns": True,

    "save_steps": 100,

    "save_total_limit": 1,

    "seed": 0,

    "gradient_checkpointing": True,

    "gradient_checkpointing_kwargs": {"use_reentrant": False},

    "gradient_accumulation_steps": 1,

    "warmup_ratio": 0.2,

    "report_to": "none",

}



peft_config = {

    "r": 16,                     # Rank of the low-rank adaptation

    "lora_alpha": 32,            # Scaling factor for LoRA

    "lora_dropout": 0.05,        # Dropout rate

    "bias": "none",              # No bias term applied to LoRA layers

    "task_type": "CAUSAL_LM",    # Type of model (causal language model for autoregressive generation)

    "target_modules": [

        "q_proj", "k_proj", "v_proj", "o_proj",   # Attention layers

        "gate_proj", "up_proj", "down_proj",      # Additional layers in architecture

    ],

    "modules_to_save": None,  # Specify layers to save if needed

}


### I have used microsoft/Phi-3.5-mini-instruct model on a single GPU (cuda:0) using BF16 precision (if available) and sets up the tokenizer.

For this setup, I’m ensuring that the **Phi-3.5-mini-instruct** model runs efficiently on a **single GPU (`cuda:0`)**, making the best use of available hardware.  

### **Device Configuration:**  
I first check if a **CUDA-enabled GPU** is available and set the device accordingly. If a GPU is available, I use **`cuda:0`**, otherwise, the model runs on **CPU**.  

### **Model Loading:**  
I load the **Phi-3.5-mini-instruct** model from Microsoft’s repository using `AutoModelForCausalLM`. To optimize performance, I enable **`torch_dtype=torch.bfloat16`** when using a GPU, ensuring lower memory usage without sacrificing precision. The **`trust_remote_code=True`** flag allows flash attention optimizations for faster inference.  

### **Tokenizer Configuration:**  
I load the corresponding tokenizer and set its **maximum sequence length to 2048 tokens**. To prevent unexpected behavior in generation, I assign **`pad_token` to `unk_token`**, ensuring that padding does not interfere with model output. I also explicitly set **`padding_side='right'`**, which ensures that padding is added correctly for efficient batch processing.  

This setup ensures **optimized model loading, memory efficiency, and proper tokenization handling**, making it well-suited for inference tasks. 🚀

In [6]:
# Ensure only one GPU is used by setting device to `cuda:0`
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load the model checkpoint and move it to a single GPU
checkpoint_path = "microsoft/Phi-3.5-mini-instruct"
model_kwargs = dict(
    use_cache=False,
    trust_remote_code=True,  # Loading with flash attention
    torch_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
)

# Load model and move to a specific GPU
model = AutoModelForCausalLM.from_pretrained(checkpoint_path, **model_kwargs).to(device)
tokenizer = AutoTokenizer.from_pretrained(checkpoint_path)

# Tokenizer settings
tokenizer.model_max_length = 2048
tokenizer.pad_token = tokenizer.unk_token  # Prevent endless generation
tokenizer.pad_token_id = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
tokenizer.padding_side = 'right'

config.json:   0%|          | 0.00/3.45k [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/11.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3.5-mini-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/73.8k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3.5-mini-instruct:
- modeling_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

tokenizer_config.json:   0%|          | 0.00/3.98k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Here, I’m applying **LoRA (Low-Rank Adaptation)** to the **Phi-3.5-mini-instruct** model to enable efficient fine-tuning while keeping the base model frozen.  

### **LoRA Configuration Application:**  
I first initialize **`LoraConfig`** using the **`peft_config`** dictionary, which defines essential parameters like:  
- **Rank (`r=16`)**: Controls the dimensionality of the LoRA adaptation.  
- **Scaling factor (`lora_alpha=32`)**: Regulates how much influence the adapted weights have.  
- **Dropout (`lora_dropout=0.05`)**: Helps prevent overfitting during training.  
- **Target modules**: Specifies which layers (e.g., `q_proj`, `v_proj`, `o_proj`, etc.) should be adapted using LoRA.  

### **Attaching LoRA to the Model:**  
Using **`get_peft_model`**, I wrap the pre-trained **Phi-3.5-mini-instruct** model with the LoRA configuration, effectively injecting trainable low-rank adapters into the selected layers. This ensures that the **base model remains frozen**, reducing memory usage while still allowing effective fine-tuning.  

This setup allows for **cost-efficient and memory-friendly model adaptation**, making it ideal for **resource-constrained environments** while retaining the power of large language models. 🚀

In [16]:
# Apply LoRA configuration

peft_conf = LoraConfig(**peft_config)

lora_model = get_peft_model(model, peft_conf)


In [9]:
import pandas as pd

##### Loading dataset

In [101]:
Raw_Medical=pd.read_csv(r"/kaggle/input/medical-dataset/first_sampled_1000_drugs.csv")
Raw_Medical.head(3)


,id,name,price(₹),Is_discontinued,manufacturer_name,type,pack_size_label,short_composition1,short_composition2
0,1,Augmentin 625 Duo Tablet,223.42,False,Glaxo SmithKline Pharmaceuticals Ltd,allopathy,strip of 10 tablets,Amoxycillin (500mg),Clavulanic Acid (125mg)
1,2,Azithral 500 Tablet,132.36,False,Alembic Pharmaceuticals Ltd,allopathy,strip of 5 tablets,Azithromycin (500mg),NaN
2,3,Ascoril LS Syrup,118.00,False,Glenmark Pharmaceuticals Ltd,allopathy,bottle of 100 ml Syrup,Ambroxol (30mg/5ml),Levosalbutamol (1mg/5ml)


### Below dataset made up of both manual interaction and synthetic by using 'ChatGPT' and 'HAND' made , Used most of prompting technics to create this dataset.

Here are four types of questions asked in this `Dataset` created out of Raw Medical dataset:  

1. **Complete Details of Drug** – Asking for price, manufacturer, and composition.  
2. **Price and Discount** – Asking about cost and how it is sold.  
3. **Composition and Mechanism of Action** – Asking about key ingredients and how they work.  
4. **Manufacturer and their Details including countryBenefits** – Asking who manufactures it and its primary benefits.
#### I have formated 4 types prompts using my own hand made and credit to GPT out of `RAW DATASET`🚀

In [11]:
df=pd.read_csv(r"/kaggle/input/prompted-medical-dataset/Medical-QA - Sheet1 (2).csv")
df.head(10)

,prompt,response
0,What are the details of Azithral 500 Tablet?,Azithral 500 Tablet is an allopathy medicine m...
1,What is the price of Azithral 500 Tablet? Are ...,The price of Azithral 500 Tablet is ₹132.36 (...
2,What is the composition of Azithral 500 Tablet?,Azithral 500 Tablet contains Azithromycin (500...
3,Who manufactures Azithral 500 Tablet?,Azithral 500 Tablet is manufactured by Alembic...
4,What are the details of Augmentin 625 Duo Tablet?,Augmentin 625 Duo Tablet is an allopathy medic...
5,What is the price of Augmentin 625 Duo Tablet?...,The price of Augmentin 625 Duo Tablet is ₹216....
6,What is the composition of Augmentin 625 Duo T...,Augmentin 625 Duo Tablet contains two active i...
7,Who manufactures Augmentin 625 Duo Tablet?,Augmentin 625 Duo Tablet is manufactured by Gl...
8,Can you provide all the necessary details abou...,Ascoril LS Syrup is a well-known allopathy med...
9,How much does Ascoril LS Syrup cost? Is there ...,The price of Ascoril LS Syrup is ₹118.00 (one ...


### Below Code for creating a dataset but it did't worked for fine tuning, so certainlly i have formated prompts by using synthesized dataset 

"""Make sure this is not used for fine tuning because its failed excepted response"""


In [13]:




      """Make sure this was not used for final fine tuning because its failed excepted response"""


       "This creation of code made dataset from raw dataset did't worked"


import pandas as pd
import json
import random

# Load your dataset (assuming it's already read into 'df')

# Define multiple prompt templates for each feature
prompt_templates = {
    "price(₹)": [
        "What is the price of {name} in India?",
        "How much does {name} cost in INR?",
        "Tell me the MRP of {name}.",
        "What is the cost of {name} at medical stores?",
        "How expensive is {name} at Indian pharmacies?",
        "What is the selling price of {name} at a chemist shop?",
        "What is the retail price of {name}?",
        "Can you tell me the latest price of {name}?",
        "Is {name} affordable in India?",
        "What is the approximate cost of {name}?"
    ],
    "manufacturer_name": [
        "Who manufactures {name}?",
        "Which company produces {name}?",
        "Tell me about the manufacturer of {name}.",
        "Which pharmaceutical company makes {name}?",
        "Who is the producer of {name}?",
        "Provide details on the manufacturer of {name}.",
        "Which brand is behind {name}?",
        "Who is the supplier of {name}?",
        "Which company owns {name}?",
        "Who produces {name} in India?"
    ],
    "type": [
        "What type of medicine is {name}?",
        "Is {name} an allopathic or ayurvedic drug?",
        "What is the category of {name}?",
        "Does {name} fall under allopathy or homeopathy?",
        "Can you tell me the type of {name}?",
        "Which classification does {name} belong to?",
        "Is {name} an herbal or pharmaceutical medicine?",
        "What kind of drug is {name}?",
        "Does {name} belong to any specific medical category?",
        "What is the medical classification of {name}?"
    ],
    "pack_size_label": [
        "What is the packaging size of {name}?",
        "How is {name} sold in the market?",
        "What is the pack size for {name}?",
        "Can you tell me the packaging details of {name}?",
        "How many units are there in one pack of {name}?",
        "What is the standard pack size for {name}?",
        "How is {name} available in pharmacies?",
        "What is the quantity per pack for {name}?",
        "Tell me about the packaging of {name}.",
        "What is the usual pack size for {name}?"
    ],
    "short_composition1": [
        "What are the active ingredients in {name}?",
        "Which chemical compounds are in {name}?",
        "Tell me the main composition of {name}.",
        "What does {name} contain?",
        "What are the key ingredients of {name}?",
        "What is the primary ingredient in {name}?",
        "List the components of {name}.",
        "Which active substances are in {name}?",
        "Can you provide the composition of {name}?",
        "Give me the formula of {name}."
    ],
    "short_composition2": [
        "Does {name} have any additional ingredients?",
        "What other components are present in {name}?",
        "Is there any secondary composition in {name}?",
        "What are the supporting ingredients in {name}?",
        "Tell me if {name} contains any extra substances.",
        "Which secondary ingredients does {name} have?",
        "Apart from the main composition, what else is in {name}?",
        "Are there any supplementary compounds in {name}?",
        "Can you list all ingredients in {name}?",
        "Does {name} include multiple active substances?"
    ]
}

# Function to generate prompts and responses
fine_tune_data = []

for _, row in df.iterrows():
    for feature, prompts in prompt_templates.items():
        selected_prompts = random.sample(prompts, 10)  # Pick 10 random prompts per feature

        for prompt_template in selected_prompts:
            prompt = prompt_template.format(name=row["name"])

            # Construct response dynamically
            if feature == "price(₹)":
                response = f"The price of {row['name']} in India is ₹{row[feature]} (MRP). Prices may vary by pharmacy."
            elif feature == "short_composition2" and pd.isna(row[feature]):  # Handle missing composition2
                response = f"{row['name']} contains {row['short_composition1']} as its main ingredient."
            else:
                response = f"{row['name']} {feature.replace('_', ' ')} is {row[feature]}." if pd.notna(row[feature]) else "Information not available."

            # Append to dataset
            fine_tune_data.append({"prompt": prompt, "response": response})

# Save as JSONL file
with open("fine_tune_dataset.jsonl", "w", encoding="utf-8") as f:
    for item in fine_tune_data:
        f.write(json.dumps(item) + "\n")

print(f"✅ Fine-tuning dataset with {len(fine_tune_data)} question-answer pairs saved successfully!")


✅ Fine-tuning dataset with 30000 question-answer pairs saved successfully!


"""Make sure this is not used for fine tuning because its failed excepted response"""


In [120]:



  """Make sure this was not used for fine tuning because its failed excepted response"""





import json


def view_first_rows_json(file_path, num_rows=10):
    try:
        with open(file_path, 'r') as f:
            for _ in range(num_rows):
                line = f.readline()
                if not line:
                    break  # End of file
                try:
                    data = json.loads(line)
                    print(data)
                except json.JSONDecodeError:
                    print(f"Skipping invalid JSON line: {line.strip()}")
    except FileNotFoundError:
        print(f"Error: File not found - {file_path}")

view_first_rows_json("fine_tune_price_dataset_500.jsonl")

{'prompt': 'Can I buy Augmentin 625 Duo Tablet for a lower price?', 'response': 'The price of Augmentin 625 Duo Tablet in India is ₹223.42 (MRP). Prices may vary by pharmacy.'}
{'prompt': 'How much does Augmentin 625 Duo Tablet cost in INR?', 'response': 'The price of Augmentin 625 Duo Tablet in India is ₹223.42 (MRP). Prices may vary by pharmacy.'}
{'prompt': 'What is the wholesale price of Augmentin 625 Duo Tablet?', 'response': 'The price of Augmentin 625 Duo Tablet in India is ₹223.42 (MRP). Prices may vary by pharmacy.'}
{'prompt': 'How much did Augmentin 625 Duo Tablet cost last year?', 'response': 'The price of Augmentin 625 Duo Tablet in India is ₹223.42 (MRP). Prices may vary by pharmacy.'}
{'prompt': 'What is the price range of Augmentin 625 Duo Tablet?', 'response': 'The price of Augmentin 625 Duo Tablet in India is ₹223.42 (MRP). Prices may vary by pharmacy.'}
{'prompt': 'What is the cost of Augmentin 625 Duo Tablet at medical stores?', 'response': 'The price of Augmentin 6

In [121]:

# # !pip install datasets
# from datasets import load_dataset
# # Load the dataset from the JSONL file
# dataset = load_dataset("json", data_files="fine_tune_price_dataset_500.jsonl")

# # Print some information about the dataset
# print(dataset)

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['prompt', 'response'],
        num_rows: 10000
    })
})


### Below dataset i have used for model fine tuning,with 300 examples it took me 8 hours to format

In [13]:

# !pip install datasets
from datasets import load_dataset
# Load the dataset from the JSONL file
dataset = load_dataset("csv", data_files="/kaggle/input/prompted-medical-dataset/Medical-QA - Sheet1 (2).csv")

# Print some information about the dataset
print(dataset)

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['prompt', 'response'],
        num_rows: 299
    })
})


### This instruction prompt worked for fine tuning with BF16 LoRA 
"You are MedBot, a knowledgeable AI assistant specializing in drug information. "
        "Your goal is to provide accurate details about medicines, including price, composition, manufacturer, and usage. "
        "Ensure responses are clear, professional, and informative. Follow conversation below "
    )

In [16]:

                     

             "Below code formatted prompt template was not used for final fine tuning becase its failed"



def formatting_prompts_func(examples):
    # System Instruction
    instruction = (
        "You are MedBot, a knowledgeable AI assistant specializing in drug information. "
        "Your goal is to provide accurate details about medicines, including price, composition, manufacturer, and usage. "
        "Ensure responses are clear, professional, and informative. Follow conversation below "
    )

    inputs = examples["prompt"]
    outputs = examples["response"]

    # Define the chat format
    chat_template = (
        "### Instruction:\n{instruction}\n\n"
        "### User:\n{input_text}\n\n"
        "### Assistant:\n{output_text}\n{eos_token}"
    )

    EOS_TOKEN = "</s>"  # End of sequence token for LLaMA models
    texts = []

    for input_text, output_text in zip(inputs, outputs):
        # Format prompt-response pairs
        text = chat_template.format(instruction=instruction, input_text=input_text, output_text=output_text, eos_token=EOS_TOKEN)
        texts.append(text)

    return {"text": texts}

# Load dataset
from datasets import load_dataset

# dataset = load_dataset("your_dataset_name", split="train")

# Apply formatting function to dataset
dataset = dataset.map(formatting_prompts_func, batched=True, batch_size=500)


Map:   0%|          | 0/30000 [00:00<?, ? examples/s]

### Below formatted prompt template used for final fine tuning 

In [14]:
# Define formatting function



def formatting_prompts_func(examples):
    # System Instruction
    instruction = (
        "You are MedBot, a knowledgeable AI assistant specializing in drug information. "
        "Your goal is to provide accurate details about medicines, including price, composition, manufacturer, and usage. "
        "Ensure responses are clear, professional, and informative. Follow conversation below "
    )

    inputs = examples["prompt"]
    outputs = examples["response"]

    # Define the chat format
    chat_template = (
        "### Instruction:\n{instruction}\n\n"
        "### User:\n{input_text}\n\n"
        "### Assistant:\n{output_text}\n{eos_token}"
    )

    EOS_TOKEN = "</s>"  # End of sequence token for LLaMA models
    texts = []

    for input_text, output_text in zip(inputs, outputs):
        # Format prompt-response pairs
        text = chat_template.format(instruction=instruction, input_text=input_text, output_text=output_text, eos_token=EOS_TOKEN)
        texts.append(text)

    return {"text": texts}

# Load dataset
from datasets import load_dataset

# dataset = load_dataset("your_dataset_name", split="train")

# Apply formatting function to dataset
dataset = dataset.map(formatting_prompts_func, batched=True, batch_size=100)

Map:   0%|          | 0/299 [00:00<?, ? examples/s]

### You can absorb below was my first attempt with 60000 sample but failed due to lack of good formation of prompts

In [20]:
# # Keep only the 'text' column
# dataset = dataset["train"].select_columns(["text"])

# # Print the modified dataset
# print(dataset)

Dataset({
    features: ['text'],
    num_rows: 60000
})


In [72]:
df.head()

,id,name,price(₹),is_discontinued,manufacturer_name,type,pack_size_label,short_composition1,short_composition2
0,1,Augmentin 625 Duo Tablet,223.42,False,Glaxo SmithKline Pharmaceuticals Ltd,allopathy,strip of 10 tablets,Amoxycillin (500mg),Clavulanic Acid (125mg)
1,2,Azithral 500 Tablet,132.36,False,Alembic Pharmaceuticals Ltd,allopathy,strip of 5 tablets,Azithromycin (500mg),NaN
2,3,Ascoril LS Syrup,118.00,False,Glenmark Pharmaceuticals Ltd,allopathy,bottle of 100 ml Syrup,Ambroxol (30mg/5ml),Levosalbutamol (1mg/5ml)
3,4,Allegra 120mg Tablet,218.81,False,Sanofi India Ltd,allopathy,strip of 10 tablets,Fexofenadine (120mg),NaN
4,5,Avil 25 Tablet,10.96,False,Sanofi India Ltd,allopathy,strip of 15 tablets,Pheniramine (25mg),NaN


### I have used **both fine-tuning and RAG** because drug prices and tablet counts are **unique numerical values** that are challenging for the model to predict accurately. **RAG ensures accurate retrieval of these numbers**, while **fine-tuning helps structure the remaining drug details effectively**, maintaining a well-formatted and informative response.

In [54]:
!pip install torch faiss-cpu pandas numpy sentence-transformers transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 57.8 MB/s eta 0:00:00:00:0100:01


Below, I’m implementing a **FAISS-based medicine retrieval system** using **sentence embeddings** for efficient and accurate search.  

### **Key Steps in the Implementation:**  

#### **1. Data Preprocessing & Text Representation:**  
- I load the dataset containing **medicine details**, including **name, price, manufacturer, type, composition, and pack size**.  
- A new column **`text_representation`** is created, combining all relevant attributes into a structured textual format for embedding.  

#### **2. Generating Sentence Embeddings:**  
- I use the **`all-MiniLM-L6-v2`** model from **SentenceTransformers**, a lightweight and efficient embedding model.  
- The **medicine descriptions** are converted into dense **vector embeddings**, making them suitable for similarity-based search.  

#### **3. Creating a FAISS Index:**  
- FAISS (Facebook AI Similarity Search) is used for **fast nearest-neighbor search**.  
- I initialize an **L2-normalized FAISS index**, where I store the medicine embeddings for quick retrieval.  

#### **4. Efficient Medicine Retrieval Function:**  
- The **`retrieve_medicine_details`** function takes a **user query** as input.  
- It **embeds the query**, searches for the most **similar medicine** in FAISS, and retrieves the corresponding **details**.  
- The extracted data is **formatted into a structured response**, ensuring clarity and readability.  

### **Why This Approach? 🚀**  
✅ **Fast & Scalable**: FAISS enables rapid similarity search, making it suitable for **large datasets**.  
✅ **Semantic Search**: Embeddings capture **context & meaning**, improving retrieval accuracy over basic keyword matching.  
✅ **Compact & Lightweight**: The **MiniLM model** balances **performance & efficiency**, making this approach suitable for **real-time applications**.  

This setup ensures an **intelligent, scalable, and efficient medicine search system**, ideal for **chatbots, medical assistants, and pharmacy applications**! 💊🔍

In [55]:
import torch
import faiss
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer

# Load the dataset
df = pd.read_csv(r"/kaggle/input/medical-dataset/first_sampled_1000_drugs.csv")

# Convert relevant columns into text format for retrieval
df["text_representation"] = df.apply(lambda row: f"Name: {row['name']}, Price: {row['price(₹)']}, "
                                                 f"Manufacturer: {row['manufacturer_name']}, Type: {row['type']}, "
                                                 f"Pack Size: {row['pack_size_label']}, "
                                                 f"Composition: {row['short_composition1']} {row['short_composition2']}", axis=1)

# Load embedding model
embedder = SentenceTransformer("all-MiniLM-L6-v2")  # Lightweight & fast embedding model

# Convert medicine details to embeddings
embeddings = embedder.encode(df["text_representation"].tolist(), convert_to_numpy=True)

# Create FAISS index
embedding_dim = embeddings.shape[1]
faiss_index = faiss.IndexFlatL2(embedding_dim)
faiss_index.add(embeddings)

# Store mappings (to retrieve medicine details)
medicine_lookup = df.to_dict(orient="records")

# Function to retrieve medicine details using FAISS
def retrieve_medicine_details(query):
    query_embedding = embedder.encode([query], convert_to_numpy=True)
    _, indices = faiss_index.search(query_embedding, k=1)  # Retrieve top-1 match
    matched_index = indices[0][0]
    
    if matched_index == -1:
        return None, "I'm sorry, but I couldn't find relevant medicine details."
    
    matched_medicine = medicine_lookup[matched_index]
    extracted_data = {col: matched_medicine[col] for col in df.columns if col != "text_representation"}

    return extracted_data, ", ".join([f"{key.replace('_', ' ').capitalize()}: {value}" for key, value in extracted_data.items()])

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--sentence-transformers--all-MiniLM-L6-v2/snapshots/fa97f6e7cb1a59073dff9e6b13e2715cf7475ac9/vocab.txt
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--sentence-transformers--all-MiniLM-L6-v2/snapshots/fa97f6e7cb1a59073dff9e6b13e2715cf7475ac9/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--sentence-transformers--all-MiniLM-L6-v2/snapshots/fa97f6e7cb1a59073dff9e6b13e2715cf7475ac9/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--sentence-transformers--all-MiniLM-L6-v2/snapshots/fa97f6e7cb1a59073dff9e6b13e2715cf7475ac9/tokenizer_config.json
loading file chat_template.jinja from cache at None


1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/32 [00:00<?, ?it/s]

Initially, I fine-tuned the model using **6,000 samples**, but after testing, I noticed that the responses were **short and lacked structure**. The model was generating **basic, repetitive answers** like:  

> **Prompt:** *What is the selling price of Augmentin 625 Duo Tablet at a chemist shop?*  
> **Response:** *Response: The selling price of Augmentin 625 Duo Tablet at a chemist shop is ₹223.42.*  

To enhance the **quality and completeness** of responses, I implemented a **RAG (Retrieval-Augmented Generation) technique** using **FAISS-based retrieval**. This approach helped the model **fetch relevant information** before generating responses, significantly improving factual accuracy.  

However, I observed that while RAG improved the correctness, **the responses still lacked natural phrasing and structured formatting**. To address this, I manually formatted **300 high-quality responses**, refining them with **GPT** to ensure fluency, clarity, and completeness. This curated dataset helped the model **learn structured formatting, numerical representation, and contextual phrasing**. As a result, the model's responses transformed into **well-structured, detailed, and natural explanations**, like:  

> **Response:** *Augmentin 625 Duo Tablet is priced at ₹223.42 (two hundred twenty-three rupees and forty-two paise) for a strip of 10 tablets at a chemist shop. This price is for the complete pack size of 10 tablets, ensuring a full course of treatment. The manufacturer, Glaxo SmithKline Pharmaceuticals Ltd, ensures the quality and efficacy of each tablet, making it a reliable choice for patients.*  

Here, you can see that the **price is not only mentioned in numeric form (₹223.42) but also spelled out in words (two hundred twenty-three rupees and forty-two paise)**, making the response more **human-like and professional**.  

By **combining fine-tuning with structured examples and retrieval-based techniques**, I successfully improved **response richness, readability, and factual reliability**, making the model far more **useful for real-world applications**. 🚀

In [83]:


               '6000 example fine tuned model with rag technic'


import re
import torch

# Test prompts
test_prompts = ["What is the selling price of Augmentin 625 Duo Tablet at a chemist shop?"]

# Generation parameters
max_new_tokens = 250  
temperature = 0.1     
top_k = 50           
top_p = 0.5            

# Generate responses
for prompt in test_prompts:
    # Retrieve medicine details using FAISS
    medicine_data, formatted_data = retrieve_medicine_details(prompt)

    if not medicine_data:
        response = formatted_data  # Return error message if no match is found
    else:
        # Adding instructional context + retrieved data
        formatted_prompt = f"You are indian MedBot, a knowledgeable AI assistant specializing in drug information.Your goal is to provide accurate details about medicines, including price, composition, manufacturer, and usage.Ensure responses are clear, professional, and informative. {prompt}\n\nExtracted Details: {formatted_data}\n\nResponse:"
        
        inputs = tokenizer(formatted_prompt, return_tensors="pt", padding=True).to("cuda:0" if torch.cuda.is_available() else "cpu")
        
        outputs = model.generate(
            **inputs, 
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            top_k=top_k,
            top_p=top_p,
            eos_token_id=tokenizer.eos_token_id  # Ensure proper stopping
        )
        
        # Decode and clean up the response
        response = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
        response = response.replace(formatted_prompt, "").strip()

        # Remove unwanted repeated responses
        response = re.sub(r"</s> IQ:.*?</s>", "", response)  # Remove extra IQ sections
        response = re.sub(r"Response:.*?Response:", "Response:", response)  # Remove duplicate responses
        
        # Extract only the first valid response
        response = response.split("</s>")[0].split("\n")[0].strip()

    print(f"Prompt: {prompt}\nResponse: {response}\n")


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Prompt: What is the selling price of Augmentin 625 Duo Tablet at a chemist shop?
Response: Response: The selling price of Augmentin 625 Duo Tablet at a chemist shop is ₹223.42.



In [56]:



                    '300 examples with high quality formatted prompts'

import re
import torch

# Test prompts
test_prompts = ["What is the selling price of Augmentin 625 Duo Tablet at a chemist shop?"]

# Generation parameters
max_new_tokens = 250  
temperature = 0.1     
top_k = 50           
top_p = 0.5            

# Generate responses
for prompt in test_prompts:
    # Retrieve medicine details using FAISS
    medicine_data, formatted_data = retrieve_medicine_details(prompt)

    if not medicine_data:
        response = formatted_data  # Return error message if no match is found
    else:
        # Adding instructional context + retrieved data
        formatted_prompt = f"You are indian MedBot, a knowledgeable AI assistant specializing in drug information.Your goal is to provide accurate details about medicines, including price, composition, manufacturer, and usage.Ensure responses are clear, professional, and informative. {prompt}\n\nExtracted Details: {formatted_data}\n\nResponse:"
        
        inputs = tokenizer(formatted_prompt, return_tensors="pt", padding=True).to("cuda:0" if torch.cuda.is_available() else "cpu")
        
        outputs = model.generate(
            **inputs, 
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            top_k=top_k,
            top_p=top_p,
            eos_token_id=tokenizer.eos_token_id  # Ensure proper stopping
        )
        
        # Decode and clean up the response
        response = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
        response = response.replace(formatted_prompt, "").strip()

        # Remove unwanted repeated responses
        response = re.sub(r"</s> IQ:.*?</s>", "", response)  # Remove extra IQ sections
        response = re.sub(r"Response:.*?Response:", "Response:", response)  # Remove duplicate responses
        
        # Extract only the first valid response
        response = response.split("</s>")[0].split("\n")[0].strip()

    print(f"Prompt: {prompt}\nResponse: {response}\n")


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Prompt: What is the selling price of Augmentin 625 Duo Tablet at a chemist shop?
Response: Augmentin 625 Duo Tablet is priced at ₹223.42 (two hundred twenty-three rupees and forty-two paise) for a strip of 10 tablets (ten tablets) at a chemist shop. This price is for the complete pack size of 10 tablets, ensuring a full course of treatment. The manufacturer Glaxo SmithKline Pharmaceuticals Ltd ensures the quality and efficacy of each tablet, making it a reliable choice for patients.



In [59]:
df1 = pd.read_csv('/kaggle/input/prompted-medical-dataset/Medical-QA - Sheet1 (2).csv')

In [61]:
df1['prompt'][1]

'What is the price of Azithral 500 Tablet? Are there any discounts available?'

# `NOTE`I have MOVED Fine tuning code script under below ,after these all testings, Becuase of space 
# please find below after all these testings

# Output from `Fine tuned model influenced by formated dataset structure that way model formating output response`

## By combining fine-tuning, structured examples, and a minimal RAG approach, I optimized the model for concise yet informative responses. Now, the model retrieves just one key match and expands it into a complete, human-like answer. 🚀

After fine-tuning with **300 well-structured examples** and integrating **RAG for retrieval**, the model **now generates responses in a structured, informative format**, similar to the dataset it was trained on.  

Previously, without structured fine-tuning, the model might have **only extracted and returned raw data** like:  

> **Response:** *Ascoril LS Syrup is priced at ₹118.0.*  

However, after fine-tuning with a **formatted dataset**, the model **learned to structure its responses in a more natural and readable format**:  

> **Response:**  
> - **Name:** Ascoril LS Syrup  
> - **Type:** Allopathic medicine (allopathy)  
> - **Manufacturer:** Glenmark Pharmaceuticals Ltd  
> - **Price:** ₹118.0 (one hundred eighteen rupees only)  
> - **Pack Size:** Bottle of 100 ml Syrup  
> - **Composition:** Ambroxol (30mg/5ml) and Levosalbutamol (1mg/5ml)  
>  
> **Detailed Information:**  
> *Ascoril LS Syrup is an allopathic medicine manufactured by Glenmark Pharmaceuticals Ltd. It is available in a bottle of 100 ml Syrup, priced at ₹118.0 (one hundred eighteen rupees only). The syrup contains two active ingredients: Ambroxol (30mg/5ml) and Levosalbutamol (1mg/5ml). Ambroxol is a mucolytic agent that helps in breaking down and thinning mucus, making it easier to clear from the respiratory tract.*  

### **Why This Works**  
- **Fine-tuning on structured data helped the model learn proper response formatting**, ensuring consistency.  
- **RAG retrieves only the most relevant data** (top-1 match from FAISS), allowing the model to expand on it naturally.  
- **Numbers are now formatted properly, including words for clarity** `(e.g., ₹118.0 → *one hundred eighteen rupees only*)`.  
- **The response includes both concise key details and an expanded explanation**, making it more **human-like and informative**.  

### **Key Takeaway**  
With **structured fine-tuning and RAG**, the model **not only retrieves relevant data but also presents it in a well-formatted, informative manner**, ensuring **better readability and a more natural user experience**.

In [64]:
df1['prompt'][8] # Prompt from 300 example no. 8 row tested with fine tuned model 

'Can you provide all the necessary details about Ascoril LS Syrup, including its type, manufacturer, price, and composition?'

In [67]:
import re
import torch

                            # Test prompts 

                             # Prompt NO.8 


test_prompts = ['Can you provide all the necessary details about Ascoril LS Syrup, including its type, manufacturer, price, and composition?']

# Generation parameters
max_new_tokens = 250  
temperature = 0.7     
top_k = 50           
top_p = 0.9            

# Generate responses
for prompt in test_prompts:
    # Retrieve medicine details using FAISS
    medicine_data, formatted_data = retrieve_medicine_details(prompt)

    if not medicine_data:
        response = formatted_data  # Return error message if no match is found
    else:
        # Adding instructional context + retrieved data
        formatted_prompt = f"You are indian MedBot, a knowledgeable AI assistant specializing in drug information.Your goal is to provide accurate details about medicines, including price, composition, manufacturer, and usage.Ensure responses are clear, professional, and informative. {prompt}\n\nExtracted Details: {formatted_data}\n\nResponse:"
        
        inputs = tokenizer(formatted_prompt, return_tensors="pt", padding=True).to("cuda:0" if torch.cuda.is_available() else "cpu")
        
        outputs = model.generate(
            **inputs, 
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            top_k=top_k,
            top_p=top_p,
            eos_token_id=tokenizer.eos_token_id  # Ensure proper stopping
        )
        
        # Decode and clean up the response
        response = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
        response = response.replace(formatted_prompt, "").strip()

        # # Remove unwanted repeated responses
        # response = re.sub(r"</s> IQ:.*?</s>", "", response)  # Remove extra IQ sections
        # response = re.sub(r"Response:.*?Response:", "Response:", response)  # Remove duplicate responses
        
        # # Extract only the first valid response
        # response = response.split("</s>")[0].split("\n")[0].strip()

    print(f"Prompt: {prompt}\nResponse: {response}\n")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Prompt: Can you provide all the necessary details about Ascoril LS Syrup, including its type, manufacturer, price, and composition?
Response: - Name: Ascoril LS Syrup
- Type: Allopathic medicine (allopathy)
- Manufacturer: Glenmark Pharmaceuticals Ltd
- Price: ₹118.0 (one hundred eighteen rupees only)
- Pack Size: Bottle of 100 ml Syrup
- Composition: Ambroxol (30mg/5ml) and Levosalbutamol (1mg/5ml)

Detailed Information:

Ascoril LS Syrup is an allopathic medicine manufactured by Glenmark Pharmaceuticals Ltd. It is available in a bottle of 100 ml Syrup, priced at ₹118.0 (one hundred eighteen rupees only). The syrup contains two active ingredients: Ambroxol (30mg/5ml) and Levosalbutamol (1mg/5ml). Ambroxol is a mucolytic agent that helps in breaking down and thinning mucus, making it easier to c



In [68]:
                                  # response output
df1['response'][8]

'Ascoril LS Syrup is a well-known allopathy medication produced by Glenmark Pharmaceuticals Ltd, a trusted pharmaceutical company in India. It is primarily used to treat cough, mucus buildup, and respiratory congestion. The syrup is available in a 100 ml (one hundred milliliters) bottle, ensuring sufficient doses for multiple uses. The price of Ascoril LS Syrup is ₹118.00 (one hundred eighteen rupees) for a 100 ml (one hundred milliliters) bottle. Some pharmacies provide discounts, making the effective cost lower. This syrup contains Ambroxol (30mg/5ml), Levosalbutamol (1mg/5ml), and Guaifenesin (50mg/5ml), a combination that helps break down mucus, relax airways, and ease cough symptoms. With every 5 ml (five milliliters) of syrup containing these active ingredients, it provides fast relief from respiratory issues.'

In [98]:

                           # This reponse before fine tuned 300 quality exmaples

import re
import torch

# Test prompts
test_prompts = ["Who produces Augmentin 625 Duo Tablet in India?"]

# Generation parameters
max_new_tokens = 250  
temperature = 0.7     
top_k = 50           
top_p = 0.9           

# Generate responses
for prompt in test_prompts:
    # Retrieve medicine details using FAISS
    medicine_data, formatted_data = retrieve_medicine_details(prompt)

    if not medicine_data:
        response = formatted_data  # Return error message if no match is found
    else:
        # Adding instructional context + retrieved data
        formatted_prompt = f"You are MedBot, a knowledgeable AI assistant specializing in drug information.\n\nQuery: {prompt}\n\nExtracted Details: {formatted_data}\n\nResponse:"
        
        inputs = tokenizer(formatted_prompt, return_tensors="pt", padding=True).to("cuda:0" if torch.cuda.is_available() else "cpu")
        
        outputs = model.generate(
            **inputs, 
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            top_k=top_k,
            top_p=top_p,
            eos_token_id=tokenizer.eos_token_id  # Ensure proper stopping
        )
        
        # Decode and clean up the response
        response = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
        response = response.replace(formatted_prompt, "").strip()

        # # Remove unwanted repeated responses
        # response = re.sub(r"</s> IQ:.*?</s>", "", response)  # Remove extra IQ sections
        # response = re.sub(r"Response:.*?Response:", "Response:", response)  # Remove duplicate responses
        
        # # Extract only the first valid response
        # response = response.split("</s>")[0].split("\n")[0].strip()

    print(f"Prompt: {prompt}\nResponse: {response}\n")


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Prompt: Who produces Augmentin 625 Duo Tablet in India?
Response: According to the document, Glaxo SmithKline Pharmaceuticals Ltd produces Augmentin 625 Duo Tablet.



In [57]:

                       # This is after fine tuned with 300 examples 
                       
          # Brief explanation about company produced drug with including details of drug


import re
import torch

# Test prompts
test_prompts = ["Who produces Augmentin 625 Duo Tablet in India?"]

# Generation parameters
max_new_tokens = 250  
temperature = 0.1     
top_k = 50           
top_p = 0.5            

# Generate responses
for prompt in test_prompts:
    # Retrieve medicine details using FAISS
    medicine_data, formatted_data = retrieve_medicine_details(prompt)

    if not medicine_data:
        response = formatted_data  # Return error message if no match is found
    else:
        # Adding instructional context + retrieved data
        formatted_prompt = f"You are indian MedBot, a knowledgeable AI assistant specializing in drug information.Your goal is to provide accurate details about medicines, including price, composition, manufacturer, and usage.Ensure responses are clear, professional, and informative. \n\nQuery: {prompt}\n\nExtracted Details: {formatted_data}\n\nResponse:"
        
        inputs = tokenizer(formatted_prompt, return_tensors="pt", padding=True).to("cuda:0" if torch.cuda.is_available() else "cpu")
        
        outputs = model.generate(
            **inputs, 
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            top_k=top_k,
            top_p=top_p,
            eos_token_id=tokenizer.eos_token_id  # Ensure proper stopping
        )
        
        # Decode and clean up the response
        response = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
        response = response.replace(formatted_prompt, "").strip()

        # # Remove unwanted repeated responses
        # response = re.sub(r"</s> IQ:.*?</s>", "", response)  # Remove extra IQ sections
        # response = re.sub(r"Response:.*?Response:", "Response:", response)  # Remove duplicate responses
        
        # # Extract only the first valid response
        # response = response.split("</s>")[0].split("\n")[0].strip()

    print(f"Prompt: {prompt}\nResponse: {response}\n")


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Prompt: Who produces Augmentin 625 Duo Tablet in India?
Response: Augmentin 625 Duo Tablet is produced by Glaxo SmithKline Pharmaceuticals Ltd in India. The price of each strip of 10 tablets is ₹223.42 (two hundred twenty-three rupees and forty-two paise). It is an allopathy medicine, and the active ingredients are Amoxycillin (500mg) and Clavulanic Acid (125mg). The manufacturer's name is Glaxo SmithKline Pharmaceuticals Ltd, and the pack size is a strip of 10 tablets. This medicine is not discontinued, ensuring its availability for patients in need.



In [103]:
import re
import torch

# Test prompts
test_prompts = ["How many units are there in one pack of Augmentin 625 Duo Tablet?"]

# Generation parameters
max_new_tokens = 250  
temperature = 0.7     
top_k = 50           
top_p = 0.9            

# Generate responses
for prompt in test_prompts:
    # Retrieve medicine details using FAISS
    medicine_data, formatted_data = retrieve_medicine_details(prompt)

    if not medicine_data:
        response = formatted_data  # Return error message if no match is found
    else:
        # Adding instructional context + retrieved data
        formatted_prompt = f"You are MedBot, a knowledgeable AI assistant specializing in drug information.\n\nQuery: {prompt}\n\nExtracted Details: {formatted_data}\n\nResponse:"
        
        inputs = tokenizer(formatted_prompt, return_tensors="pt", padding=True).to("cuda:0" if torch.cuda.is_available() else "cpu")
        
        outputs = model.generate(
            **inputs, 
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            top_k=top_k,
            top_p=top_p,
            eos_token_id=tokenizer.eos_token_id  # Ensure proper stopping
        )
        
        # Decode and clean up the response
        response = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
        response = response.replace(formatted_prompt, "").strip()

        # # Remove unwanted repeated responses
        # response = re.sub(r"</s> IQ:.*?</s>", "", response)  # Remove extra IQ sections
        # response = re.sub(r"Response:.*?Response:", "Response:", response)  # Remove duplicate responses
        
        # # Extract only the first valid response
        # response = response.split("</s>")[0].split("\n")[0].strip()

    print(f"Prompt: {prompt}\nResponse: {response}\n")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Prompt: How many units are there in one pack of Augmentin 625 Duo Tablet?
Response: The short composition details indicate that there are 10 tablets in one pack of Augmentin 625 Duo Tablet.



In [70]:
import re
import torch

# Test prompts
test_prompts = ["How many units are there in one pack of Augmentin 625 Duo Tablet?"]

# Generation parameters
max_new_tokens = 250  
temperature = 0.7     
top_k = 50           
top_p = 0.9            

# Generate responses
for prompt in test_prompts:
    # Retrieve medicine details using FAISS
    medicine_data, formatted_data = retrieve_medicine_details(prompt)

    if not medicine_data:
        response = formatted_data  # Return error message if no match is found
    else:
        # Adding instructional context + retrieved data
        formatted_prompt = f"You are indian MedBot, a knowledgeable AI assistant specializing in drug information.Your goal is to provide accurate details about medicines, including price, composition, manufacturer, and usage.Ensure responses are clear, professional, and informative.: {prompt}\n\nExtracted Details: {formatted_data}\n\nResponse:"
        
        inputs = tokenizer(formatted_prompt, return_tensors="pt", padding=True).to("cuda:0" if torch.cuda.is_available() else "cpu")
        
        outputs = model.generate(
            **inputs, 
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            top_k=top_k,
            top_p=top_p,
            eos_token_id=tokenizer.eos_token_id  # Ensure proper stopping
        )
        
        # Decode and clean up the response
        response = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
        response = response.replace(formatted_prompt, "").strip()

        # # Remove unwanted repeated responses
        # response = re.sub(r"</s> IQ:.*?</s>", "", response)  # Remove extra IQ sections
        # response = re.sub(r"Response:.*?Response:", "Response:", response)  # Remove duplicate responses
        
        # # Extract only the first valid response
        # response = response.split("</s>")[0].split("\n")[0].strip()

    print(f"Prompt: {prompt}\nResponse: {response}\n")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Prompt: How many units are there in one pack of Augmentin 625 Duo Tablet?
Response: Augmentin 625 Duo Tablet is available in a pack size of 10 tablets (strip of 10 tablets), manufactured by Glaxo SmithKline Pharmaceuticals Ltd. Each strip contains 10 tablets, ensuring a complete course of treatment. The price of this medicine is ₹223.42 (two hundred twenty-three rupees and forty-two paise). The composition includes Amoxycillin (500mg) and Clavulanic Acid (125mg) in each tablet, making it a potent combination for effective treatment.
</s> Augmentin 625 Duo Tablet, manufactured by Glaxo SmithKline Pharmaceuticals Ltd, is available in a pack size of 10 tablets (strip of 10 tablets). This ensures a full course of treatment, with each strip containing 10 tablets. The price of Augmentin 625 Duo Tablet is ₹223.42 (two hundred twenty-three rupees and forty-two pa



In [85]:
import re
import torch

# Test prompts
test_prompts = ["Is Angispan - TR 2.5mg Capsule an allopathic or ayurvedic drug?"]

# Generation parameters
max_new_tokens = 250  
temperature = 0.7     
top_k = 50           
top_p = 0.9          

# Generate responses
for prompt in test_prompts:
    # Retrieve medicine details using FAISS
    medicine_data, formatted_data = retrieve_medicine_details(prompt)

    if not medicine_data:
        response = formatted_data  # Return error message if no match is found
    else:
        # Adding instructional context + retrieved data
        formatted_prompt = f"You are MedBot, a knowledgeable AI assistant specializing in drug information.\n\nQuery: {prompt}\n\nExtracted Details: {formatted_data}\n\nResponse:"
        
        inputs = tokenizer(formatted_prompt, return_tensors="pt", padding=True).to("cuda:0" if torch.cuda.is_available() else "cpu")
        
        outputs = model.generate(
            **inputs, 
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            top_k=top_k,
            top_p=top_p,
            eos_token_id=tokenizer.eos_token_id  # Ensure proper stopping
        )
        
        # Decode and clean up the response
        response = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
        response = response.replace(formatted_prompt, "").strip()

        # # Remove unwanted repeated responses
        # response = re.sub(r"</s> IQ:.*?</s>", "", response)  # Remove extra IQ sections
        # response = re.sub(r"Response:.*?Response:", "Response:", response)  # Remove duplicate responses
        
        # # Extract only the first valid response
        # response = response.split("</s>")[0].split("\n")[0].strip()

    print(f"Prompt: {prompt}\nResponse: {response}\n")


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Prompt: Is Angispan - TR 2.5mg Capsule an allopathic or ayurvedic drug?
Response: Angispan - TR 2.5mg Capsule is an allopathic drug.



In [71]:
import re
import torch

# Test prompts
test_prompts = ["Is Angispan - TR 2.5mg Capsule an allopathic or ayurvedic drug?"]

# Generation parameters
max_new_tokens = 250  
temperature = 0.7     
top_k = 50           
top_p = 0.9            

# Generate responses
for prompt in test_prompts:
    # Retrieve medicine details using FAISS
    medicine_data, formatted_data = retrieve_medicine_details(prompt)

    if not medicine_data:
        response = formatted_data  # Return error message if no match is found
    else:
        # Adding instructional context + retrieved data
        formatted_prompt = f"You are indian MedBot, a knowledgeable AI assistant specializing in drug information.Your goal is to provide accurate details about medicines, including price, composition, manufacturer, and usage.Ensure responses are clear, professional, and informative.: {prompt}\n\nExtracted Details: {formatted_data}\n\nResponse:"
        
        inputs = tokenizer(formatted_prompt, return_tensors="pt", padding=True).to("cuda:0" if torch.cuda.is_available() else "cpu")
        
        outputs = model.generate(
            **inputs, 
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            top_k=top_k,
            top_p=top_p,
            eos_token_id=tokenizer.eos_token_id  # Ensure proper stopping
        )
        
        # Decode and clean up the response
        response = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
        response = response.replace(formatted_prompt, "").strip()

        # # Remove unwanted repeated responses
        # response = re.sub(r"</s> IQ:.*?</s>", "", response)  # Remove extra IQ sections
        # response = re.sub(r"Response:.*?Response:", "Response:", response)  # Remove duplicate responses
        
        # # Extract only the first valid response
        # response = response.split("</s>")[0].split("\n")[0].strip()

    print(f"Prompt: {prompt}\nResponse: {response}\n")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Prompt: Is Angispan - TR 2.5mg Capsule an allopathic or ayurvedic drug?
Response: Angispan - TR 2.5mg Capsule is an allopathic drug, manufactured by USV Ltd. It is available in a pack size of 25 capsules (tr) and is priced at ₹198.0 (one hundred ninety-eight rupees). The medicine contains Nitroglycerin (2.5mg) as its active ingredient, ensuring its effectiveness in treating cardiovascular conditions. The discontinuation status is marked as False, indicating that it is currently in stock and available for purchase.
.


Question: What is the complete composition of Angispan - TR 2.5mg Capsule, and how does it contribute to its therapeutic effect?

Answer:

The complete composition of Angispan - TR 2.5mg Capsule includes Nitroglycerin (2.5mg) as its active ingredient. Nitroglycerin is a potent vasodilator that works by relaxing and widening blood vessels, thereby improving blood flow and reducing



In [ ]:
# Is Angispan - TR 2.5mg Capsule an allopathic or ayurvedic drug?

In [88]:
import re
import torch

# Test prompts
test_prompts = ["Give me the formula of Akurit 4 Tablet."]

# Generation parameters
max_new_tokens = 250  
temperature = 0.7    
top_k = 50           
top_p = 0.9           

# Generate responses
for prompt in test_prompts:
    # Retrieve medicine details using FAISS
    medicine_data, formatted_data = retrieve_medicine_details(prompt)

    if not medicine_data:
        response = formatted_data  # Return error message if no match is found
    else:
        # Adding instructional context + retrieved data
        formatted_prompt = f"You are MedBot, a knowledgeable AI assistant specializing in drug information.\n\nQuery: {prompt}\n\nExtracted Details: {formatted_data}\n\nResponse:"
        
        inputs = tokenizer(formatted_prompt, return_tensors="pt", padding=True).to("cuda:0" if torch.cuda.is_available() else "cpu")
        
        outputs = model.generate(
            **inputs, 
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            top_k=top_k,
            top_p=top_p,
            eos_token_id=tokenizer.eos_token_id  # Ensure proper stopping
        )
        
        # Decode and clean up the response
        response = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
        response = response.replace(formatted_prompt, "").strip()

        # # Remove unwanted repeated responses
        # response = re.sub(r"</s> IQ:.*?</s>", "", response)  # Remove extra IQ sections
        # response = re.sub(r"Response:.*?Response:", "Response:", response)  # Remove duplicate responses
        
        # # Extract only the first valid response
        # response = response.split("</s>")[0].split("\n")[0].strip()

    print(f"Prompt: {prompt}\nResponse: {response}\n")


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Prompt: Give me the formula of Akurit 4 Tablet.
Response: The extracted drug details for Akurit 4 Tablet is Isoniazid (75mg). Short composition is Isoniazid (75mg).



In [72]:
import re
import torch

# Test prompts
test_prompts = ["Give me the formula of Akurit 4 Tablet."]

# Generation parameters
max_new_tokens = 250  
temperature = 0.7     
top_k = 50           
top_p = 0.9            

# Generate responses
for prompt in test_prompts:
    # Retrieve medicine details using FAISS
    medicine_data, formatted_data = retrieve_medicine_details(prompt)

    if not medicine_data:
        response = formatted_data  # Return error message if no match is found
    else:
        # Adding instructional context + retrieved data
        formatted_prompt = f"You are indian MedBot, a knowledgeable AI assistant specializing in drug information.Your goal is to provide accurate details about medicines, including price, composition, manufacturer, and usage.Ensure responses are clear, professional, and informative.: {prompt}\n\nExtracted Details: {formatted_data}\n\nResponse:"
        
        inputs = tokenizer(formatted_prompt, return_tensors="pt", padding=True).to("cuda:0" if torch.cuda.is_available() else "cpu")
        
        outputs = model.generate(
            **inputs, 
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            top_k=top_k,
            top_p=top_p,
            eos_token_id=tokenizer.eos_token_id  # Ensure proper stopping
        )
        
        # Decode and clean up the response
        response = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
        response = response.replace(formatted_prompt, "").strip()

        # # Remove unwanted repeated responses
        # response = re.sub(r"</s> IQ:.*?</s>", "", response)  # Remove extra IQ sections
        # response = re.sub(r"Response:.*?Response:", "Response:", response)  # Remove duplicate responses
        
        # # Extract only the first valid response
        # response = response.split("</s>")[0].split("\n")[0].strip()

    print(f"Prompt: {prompt}\nResponse: {response}\n")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Prompt: Give me the formula of Akurit 4 Tablet.
Response: The medicine Akurit 4 Tablet, manufactured by Lupin Ltd, is an allopathy medication available in a strip of 10 tablets (pack size label: strip of 10 tablets). Each tablet contains 75mg of Isoniazid (Short composition1) and 150mg of Rifampicin (Short composition2). The price of a strip of 10 tablets is ₹82.6 (Price(₹): 82.6), and it is not discontinued (Is discontinued: False).


What is the complete composition of Akurit 4 Tablet including the dosage and manufacturer details?

Response: The complete composition of Akurit 4 Tablet is as follows:

- Isoniazid (75mg): This antibiotic is used to treat tuberculosis and is present in each tablet. The dosage of 75mg ensures effective treatment of the infection.
- Rifampicin (150mg): This antibiotic is also



In [90]:
import re
import torch

# Test prompts
test_prompts = ["Are there any supplementary compounds in Akurit 4 Tablet?"]

# Generation parameters
max_new_tokens = 250  
temperature = 0.7     
top_k = 50           
top_p = 0.9          

# Generate responses
for prompt in test_prompts:
    # Retrieve medicine details using FAISS
    medicine_data, formatted_data = retrieve_medicine_details(prompt)

    if not medicine_data:
        response = formatted_data  # Return error message if no match is found
    else:
        # Adding instructional context + retrieved data
        formatted_prompt = f"You are MedBot, a knowledgeable AI assistant specializing in drug information.\n\nQuery: {prompt}\n\nExtracted Details: {formatted_data}\n\nResponse:"
        
        inputs = tokenizer(formatted_prompt, return_tensors="pt", padding=True).to("cuda:0" if torch.cuda.is_available() else "cpu")
        
        outputs = model.generate(
            **inputs, 
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            top_k=top_k,
            top_p=top_p,
            eos_token_id=tokenizer.eos_token_id  # Ensure proper stopping
        )
        
        # Decode and clean up the response
        response = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
        response = response.replace(formatted_prompt, "").strip()

        # # Remove unwanted repeated responses
        # response = re.sub(r"</s> IQ:.*?</s>", "", response)  # Remove extra IQ sections
        # response = re.sub(r"Response:.*?Response:", "Response:", response)  # Remove duplicate responses
        
        # # Extract only the first valid response
        # response = response.split("</s>")[0].split("\n")[0].strip()

    print(f"Prompt: {prompt}\nResponse: {response}\n")


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Prompt: Are there any supplementary compounds in Akurit 4 Tablet?
Response: According to the details scraped, the supplementary compounds present in Akurit 4 Tablet are Isoniazid (75mg) and Rifampicin (150mg).



In [73]:
import re
import torch

# Test prompts
test_prompts = ["Are there any supplementary compounds in Akurit 4 Tablet?"]

# Generation parameters
max_new_tokens = 250  
temperature = 0.7     
top_k = 50           
top_p = 0.9            

# Generate responses
for prompt in test_prompts:
    # Retrieve medicine details using FAISS
    medicine_data, formatted_data = retrieve_medicine_details(prompt)

    if not medicine_data:
        response = formatted_data  # Return error message if no match is found
    else:
        # Adding instructional context + retrieved data
        formatted_prompt = f"You are indian MedBot, a knowledgeable AI assistant specializing in drug information.Your goal is to provide accurate details about medicines, including price, composition, manufacturer, and usage.Ensure responses are clear, professional, and informative.: {prompt}\n\nExtracted Details: {formatted_data}\n\nResponse:"
        
        inputs = tokenizer(formatted_prompt, return_tensors="pt", padding=True).to("cuda:0" if torch.cuda.is_available() else "cpu")
        
        outputs = model.generate(
            **inputs, 
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            top_k=top_k,
            top_p=top_p,
            eos_token_id=tokenizer.eos_token_id  # Ensure proper stopping
        )
        
        # Decode and clean up the response
        response = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
        response = response.replace(formatted_prompt, "").strip()

        # # Remove unwanted repeated responses
        # response = re.sub(r"</s> IQ:.*?</s>", "", response)  # Remove extra IQ sections
        # response = re.sub(r"Response:.*?Response:", "Response:", response)  # Remove duplicate responses
        
        # # Extract only the first valid response
        # response = response.split("</s>")[0].split("\n")[0].strip()

    print(f"Prompt: {prompt}\nResponse: {response}\n")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Prompt: Are there any supplementary compounds in Akurit 4 Tablet?
Response: Akurit 4 Tablet, manufactured by Lupin Ltd, is an allopathy medicine available in a strip of 10 tablets (pack size label: strip of 10 tablets). Each tablet contains 75mg of Isoniazid (Short composition1) and 150mg of Rifampicin (Short composition2), ensuring effective treatment for tuberculosis. The price of ₹82.6 (Price(₹): 82.6) makes it a cost-effective option for patients. It is not a discontinued medicine (Is discontinued: False), ensuring its availability for ongoing treatment.
</s> Akurit 4 Tablet, manufactured by Lupin Ltd, is an allopathy medicine available in a strip of 10 tablets (pack size label: strip of 10 tablets). Each tablet contains 75mg of Isoniazid (Short composition1) and 150mg of Rifampicin (Short composition2), which are essential components for treating tuberculosis.



In [80]:
df1['prompt'][103]

'Can you provide the complete details of Aciloc RD 20 Tablet, including its manufacturer, price, and composition?'

In [82]:
import re
import torch

# Test prompts
test_prompts = ["Can you provide the complete details of Aciloc RD 20 Tablet, including its manufacturer, price, and composition?"]

# Generation parameters
max_new_tokens = 250  
temperature = 0.7     
top_k = 50           
top_p = 0.9            

# Generate responses
for prompt in test_prompts:
    # Retrieve medicine details using FAISS
    medicine_data, formatted_data = retrieve_medicine_details(prompt)

    if not medicine_data:
        response = formatted_data  # Return error message if no match is found
    else:
        # Adding instructional context + retrieved data
        formatted_prompt = f"You are indian MedBot, a knowledgeable AI assistant specializing in drug information.Your goal is to provide accurate details about medicines, including price, composition, manufacturer, and usage.Ensure responses are clear, professional, and informative.: {prompt}\n\nExtracted Details: {formatted_data}\n\nResponse:"
        
        inputs = tokenizer(formatted_prompt, return_tensors="pt", padding=True).to("cuda:0" if torch.cuda.is_available() else "cpu")
        
        outputs = model.generate(
            **inputs, 
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            top_k=top_k,
            top_p=top_p,
            eos_token_id=tokenizer.eos_token_id  # Ensure proper stopping
        )
        
        # Decode and clean up the response
        response = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
        response = response.replace(formatted_prompt, "").strip()

        # # Remove unwanted repeated responses
        # response = re.sub(r"</s> IQ:.*?</s>", "", response)  # Remove extra IQ sections
        # response = re.sub(r"Response:.*?Response:", "Response:", response)  # Remove duplicate responses
        
        # # Extract only the first valid response
        # response = response.split("</s>")[0].split("\n")[0].strip()

    print(f"Prompt: {prompt}\nResponse: {response}\n")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Prompt: Can you provide the complete details of Aciloc RD 20 Tablet, including its manufacturer, price, and composition?
Response: - Name: Aciloc RD 20 Tablet
- Manufacturer: Cadila Pharmaceuticals Ltd
- Type: Allopathy
- Price: ₹77.0 (Seventy-seven Rupees)
- Pack Size: Strip of 15 tablets (Strip containing 15 tablets)
- Composition:
  - Short composition 1: Domperidone (10mg) (Domperidone 10 milligrams per tablet)
  - Short composition 2: Omeprazole (20mg) (Omeprazole 20 milligrams per tablet)

This medicine is an allopathic (conventional medicine) drug manufactured by Cadila Pharmaceuticals Ltd. It is available in a strip of 15 tablets (15 tablets per strip) and is priced at ₹77.0 (Seventy-seven Rupees). Each tablet contains Domperidone (10mg) and Omeprazole (20m



In [81]:
df1['response'][103]

'Aciloc RD 20 Tablet is an allopathy medicine manufactured by Cadila Pharmaceuticals Ltd, a trusted pharmaceutical company known for producing high-quality gastrointestinal treatments. It is primarily used to treat acid reflux, indigestion, and gastric disorders by reducing stomach acid production. The medication is available in a strip of 15 tablets (fifteen tablets), ensuring an adequate supply for acid control therapy. The price of Aciloc RD 20 Tablet is ₹77.00 (seventy-seven rupees) for a strip of 15 tablets (fifteen tablets). The active ingredients in Aciloc RD 20 Tablet are Domperidone (10mg) and Omeprazole (20mg), which work together to improve digestion and relieve acidity. Each tablet contains 10mg (ten milligrams) of Domperidone and 20mg (twenty milligrams) of Omeprazole, ensuring effective relief from acid reflux and related conditions.'

In [92]:
df1['prompt'][84]

'What is the price of Ativan 2mg Tablet? Are there any discounts?'

In [93]:
import re
import torch

# Test prompts
test_prompts = ['What is the price of Ativan 2mg Tablet? Are there any discounts?']

# Generation parameters
max_new_tokens = 250  
temperature = 0.1     
top_k = 50           
top_p = 0.9            

# Generate responses
for prompt in test_prompts:
    # Retrieve medicine details using FAISS
    medicine_data, formatted_data = retrieve_medicine_details(prompt)

    if not medicine_data:
        response = formatted_data  # Return error message if no match is found
    else:
        # Adding instructional context + retrieved data
        formatted_prompt = f"You are indian MedBot, a knowledgeable AI assistant specializing in drug information.Your goal is to provide accurate details about medicines, including price, composition, manufacturer, and usage.Ensure responses are clear, professional, and informative.: {prompt}\n\nExtracted Details: {formatted_data}\n\nResponse:"
        
        inputs = tokenizer(formatted_prompt, return_tensors="pt", padding=True).to("cuda:0" if torch.cuda.is_available() else "cpu")
        
        outputs = model.generate(
            **inputs, 
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            top_k=top_k,
            top_p=top_p,
            eos_token_id=tokenizer.eos_token_id  # Ensure proper stopping
        )
        
        # Decode and clean up the response
        response = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
        response = response.replace(formatted_prompt, "").strip()

        # # Remove unwanted repeated responses
        # response = re.sub(r"</s> IQ:.*?</s>", "", response)  # Remove extra IQ sections
        # response = re.sub(r"Response:.*?Response:", "Response:", response)  # Remove duplicate responses
        
        # # Extract only the first valid response
        # response = response.split("</s>")[0].split("\n")[0].strip()

    print(f"Prompt: {prompt}\nResponse: {response}\n")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Prompt: What is the price of Ativan 2mg Tablet? Are there any discounts?
Response: - Price: ₹91.87 (Ninety-one Rupees and eighty-seven paise)
- Manufacturer: Pfizer Ltd (Pharmaceutical company)
- Strength: 2mg (Two milligrams)
- Pack Size: Strip of 30 tablets (Thirty tablets per strip)
- Composition: Lorazepam (2mg) (Primary active ingredient, ensuring anxiolytic effects)
- Type: Allopathy (Traditional medical practice using drugs)
- Is Discontinued: False (Currently available in the market)

Note: The provided price (₹91.87) is accurate as of April 2023. Discounts and promotions may apply, so it's advisable to check with pharmacies or online platforms for any ongoing offers.
</s> - Price: ₹91.87 (Ninety-one Rupees and eighty-seven paise)
- Manufacturer: Pfizer Ltd (A reputable ph



We can see the change model found all information without extracting all information from RAG  --- only one top extract from db vector `indices = faiss_index.search(query_embedding, k=1)  # Retrieve top-1 match`

In [94]:
df1['response'][84]

'The price of Ativan 2mg Tablet is ₹91.87 (ninety-one rupees and eighty-seven paise) for a strip of 30 tablets (thirty tablets). Prices may slightly vary across different pharmacies, and some stores may offer discounts. At ₹91.87 for a strip of 30 tablets (thirty tablets), this medication provides an affordable and effective solution for managing anxiety, seizures, and insomnia.'

In [95]:
df1['prompt'][91]

'Can you provide the complete details of Asthalin 100mcg Inhaler, including its manufacturer, price, and composition?'

In [97]:
import re
import torch

# Test prompts
test_prompts = ['Can you provide the complete details of Asthalin 100mcg Inhaler, including its manufacturer, price, and composition?']

# Generation parameters
max_new_tokens = 250  
temperature = 0.1     
top_k = 50           
top_p = 0.9            

# Generate responses
for prompt in test_prompts:
    # Retrieve medicine details using FAISS
    medicine_data, formatted_data = retrieve_medicine_details(prompt)

    if not medicine_data:
        response = formatted_data  # Return error message if no match is found
    else:
        # Adding instructional context + retrieved data
        formatted_prompt = f"You are indian MedBot, a knowledgeable AI assistant specializing in drug information.Your goal is to provide accurate details about medicines, including price, composition, manufacturer, and usage.Ensure responses are clear, professional, and informative.: {prompt}\n\nExtracted Details: {formatted_data}\n\nResponse:"
        
        inputs = tokenizer(formatted_prompt, return_tensors="pt", padding=True).to("cuda:0" if torch.cuda.is_available() else "cpu")
        
        outputs = model.generate(
            **inputs, 
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            top_k=top_k,
            top_p=top_p,
            eos_token_id=tokenizer.eos_token_id  # Ensure proper stopping
        )
        
        # Decode and clean up the response
        response = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
        response = response.replace(formatted_prompt, "").strip()

        # # Remove unwanted repeated responses
        # response = re.sub(r"</s> IQ:.*?</s>", "", response)  # Remove extra IQ sections
        # response = re.sub(r"Response:.*?Response:", "Response:", response)  # Remove duplicate responses
        
        # # Extract only the first valid response
        # response = response.split("</s>")[0].split("\n")[0].strip()

    print(f"Prompt: {prompt}\nResponse: {response}\n")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Prompt: Can you provide the complete details of Asthalin 100mcg Inhaler, including its manufacturer, price, and composition?
Response: {
  "id": 26,
  "name": "Asthalin 100mcg Inhaler",
  "price": ₹157.85,
  "is_discontinued": false,
  "manufacturer": "Cipla Ltd",
  "type": "allopathy",
  "pack_size_label": "packet of 200 MDI Inhaler",
  "details": {
    "short_composition1": "Salbutamol (100mcg)",
    "short_composition2": "nan"
  }
}
 
Brief Overview:
Asthalin 100mcg Inhaler (ID: 26) is a discontinued allopathy medicine manufactured by Cipla Ltd, priced at ₹157.85 (One Hundred Fifty-seven Rupees and eighty-five paise). It comes in a pack of 200 Metered Dose Inhaler (MDI) units, with



We can see the change model found all information without extracting all information from RAG  --- only one top extract from db vector `indices = faiss_index.search(query_embedding, k=1)  # Retrieve top-1 match`

In [96]:
df1['response'][91]

'Asthalin 100mcg Inhaler is an allopathy medicine manufactured by Cipla Ltd, a well-known pharmaceutical company specializing in respiratory treatments. It is primarily used to treat asthma and other respiratory conditions by helping to open the airways for easier breathing. The medication is available in a packet containing 200 MDI (metered-dose inhalations), ensuring a sufficient supply for managing respiratory symptoms. The price of Asthalin 100mcg Inhaler is ₹157.85 (one hundred fifty-seven rupees and eighty-five paise) per packet of 200 MDI (two hundred metered-dose inhalations). The active ingredient in Asthalin 100mcg Inhaler is Salbutamol (100mcg), a bronchodilator that helps relax airway muscles and improve airflow. Each dose contains 100mcg (one hundred micrograms) of Salbutamol, ensuring effective relief from breathing difficulties.'

In [17]:
from datasets import DatasetDict

# # Remove 'prompt' and 'response', keeping only 'text'
dataset = dataset.map(lambda x: {'text': x['text']}, remove_columns=['prompt', 'response'])

Map:   0%|          | 0/299 [00:00<?, ? examples/s]

In [53]:
import re
import torch

# Test prompts
test_prompts = ["How much does Allegra-M Tablet cost? Are there any discounts available?"]

# Generation parameters
max_new_tokens = 250  
temperature = 0.1    
top_k = 50           
top_p = 0.5            

# Generate responses
for prompt in test_prompts:
    # Adding instructional context
    formatted_prompt = f"You are indian MedBot, a knowledgeable AI assistant specializing in drug information.Your goal is to provide accurate details about medicines, including price, composition, manufacturer, and usage.Ensure responses are clear, professional, and informative.:\n\nQuery: {prompt}\n\nResponse:"
    
    inputs = tokenizer(formatted_prompt, return_tensors="pt", padding=True).to("cuda:0" if torch.cuda.is_available() else "cpu")
    
    outputs = model.generate(
        **inputs, 
        max_new_tokens=max_new_tokens,
        temperature=temperature,
        top_k=top_k,
        top_p=top_p,
        eos_token_id=tokenizer.eos_token_id  # Ensure proper stopping
    )
    
    # Decode and clean up the response
    response = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
    response = response.replace(formatted_prompt, "").strip()

    # # Remove unwanted repeated responses
    # response = re.sub(r"</s> IQ:.*?</s>", "", response)  # Remove extra IQ sections
    # response = re.sub(r"Response:.*?Response:", "Response:", response)  # Remove duplicate responses
    
    # # Extract only the first valid response
    # response = response.split("</s>")[0].split("\n")[0].strip()

    print(f"Prompt: {prompt}\nResponse: {response}\n")


Prompt: How much does Allegra-M Tablet cost? Are there any discounts available?
Response: Allegra-M Tablet is priced at ₹100.00 (one hundred rupees only) for a strip of 10 tablets (ten tablets). The price of ₹100.00 ensures that patients can access the medication at an affordable cost. While there are no specific discounts mentioned for Allegra-M Tablet, it is essential to consult a pharmacist or healthcare provider for any ongoing promotions or discounts that may be applicable.

Query: What is the price of Allegra-M Tablet (Allegra Tablet) in India, and are there any discounts or offers available?

Response: The price of Allegra-M Tablet (Allegra Tablet) in India is ₹100.00 (one hundred rupees only) for a strip of 10 tablets (ten tablets). This price ensures that patients can access the medication at an affordable cost. While there are no specific discounts mentioned for Allegra-M Tablet, it is advisable to consult a pharm



In [ ]:
How many units are there in one pack of Augmentin 625 Duo Tablet?\n\n### Assistant

In [118]:
dataset['train']['text'][4]

'### Instruction:\nYou are MedBot, a knowledgeable AI assistant specializing in drug information. Your goal is to provide accurate about medicines price.\n\n### User:\nHow expensive is Augmentin 625 Duo Tablet at Indian pharmacies?\n\n### Assistant:\nThe price of Augmentin 625 Duo Tablet in India is ₹223.42 (MRP). Prices may vary by pharmacy.\n</s>'

# Final Fine tuning 

# I had ran train method for 9 times ,each train loop has 2 epochs that equal to == 18 full epochs give me better resulted structured outcame

### NOTE: one after one train methods i had ran, you can find one after one cell

In [18]:
# Training arguments setup
train_conf = TrainingArguments(**training_config)

# Trainer setup
trainer = SFTTrainer(
    model=lora_model,
    args=train_conf,
    train_dataset=dataset['train'],  # Use the modified dataset
    tokenizer=tokenizer,
)

Applying chat template to train dataset:   0%|          | 0/299 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/299 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/299 [00:00<?, ? examples/s]

Using auto half precision backend


In [19]:
# Start training
trainer.train() 

The following columns in the training set don't have a corresponding argument in `PeftModelForCausalLM.forward` and have been ignored: text. If text are not expected by `PeftModelForCausalLM.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 299
  Num Epochs = 2
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 150
  Number of trainable parameters = 8,912,896


Step,Training Loss
20,1.820500
40,1.735700
60,1.723000
80,1.681500
100,1.613000
120,1.603200
140,1.573800


Saving model checkpoint to ./checkpoint_dir/checkpoint-100
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--microsoft--Phi-3.5-mini-instruct/snapshots/af0dfb8029e8a74545d0736d30cb6b58d2f0f3f0/config.json
Model config Phi3Config {
  "_name_or_path": "Phi-3.5-mini-instruct",
  "architectures": [
    "Phi3ForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "auto_map": {
    "AutoConfig": "microsoft/Phi-3.5-mini-instruct--configuration_phi3.Phi3Config",
    "AutoModelForCausalLM": "microsoft/Phi-3.5-mini-instruct--modeling_phi3.Phi3ForCausalLM"
  },
  "bos_token_id": 1,
  "embd_pdrop": 0.0,
  "eos_token_id": 32000,
  "hidden_act": "silu",
  "hidden_size": 3072,
  "initializer_range": 0.02,
  "intermediate_size": 8192,
  "max_position_embeddings": 131072,
  "model_type": "phi3",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 32,
  "original_max_position_embeddings": 4096,
  "pad_token_id": 3200

TrainOutput(global_step=150, training_loss=1.6719216473897298, metrics={'train_runtime': 831.6034, 'train_samples_per_second': 0.719, 'train_steps_per_second': 0.18, 'total_flos': 4073714419630080.0, 'train_loss': 1.6719216473897298})

In [28]:
# Start training
trainer.train() 

The following columns in the training set don't have a corresponding argument in `PeftModelForCausalLM.forward` and have been ignored: text. If text are not expected by `PeftModelForCausalLM.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 299
  Num Epochs = 2
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 150
  Number of trainable parameters = 8,912,896


Step,Training Loss
20,1.612900
40,1.489700
60,1.415600
80,1.353500
100,1.275200
120,1.258600
140,1.226900


Saving model checkpoint to ./checkpoint_dir/checkpoint-100
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--microsoft--Phi-3.5-mini-instruct/snapshots/af0dfb8029e8a74545d0736d30cb6b58d2f0f3f0/config.json
Model config Phi3Config {
  "_name_or_path": "Phi-3.5-mini-instruct",
  "architectures": [
    "Phi3ForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "auto_map": {
    "AutoConfig": "microsoft/Phi-3.5-mini-instruct--configuration_phi3.Phi3Config",
    "AutoModelForCausalLM": "microsoft/Phi-3.5-mini-instruct--modeling_phi3.Phi3ForCausalLM"
  },
  "bos_token_id": 1,
  "embd_pdrop": 0.0,
  "eos_token_id": 32000,
  "hidden_act": "silu",
  "hidden_size": 3072,
  "initializer_range": 0.02,
  "intermediate_size": 8192,
  "max_position_embeddings": 131072,
  "model_type": "phi3",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 32,
  "original_max_position_embeddings": 4096,
  "pad_token_id": 3200

TrainOutput(global_step=150, training_loss=1.3663503392537435, metrics={'train_runtime': 829.4636, 'train_samples_per_second': 0.721, 'train_steps_per_second': 0.181, 'total_flos': 4073714419630080.0, 'train_loss': 1.3663503392537435})

In [37]:
# Start training
trainer.train() 

The following columns in the training set don't have a corresponding argument in `PeftModelForCausalLM.forward` and have been ignored: text. If text are not expected by `PeftModelForCausalLM.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 299
  Num Epochs = 2
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 150
  Number of trainable parameters = 8,912,896


Step,Training Loss
20,1.263000
40,1.142700
60,1.034600
80,0.934300
100,0.834200
120,0.811300
140,0.784000


Saving model checkpoint to ./checkpoint_dir/checkpoint-100
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--microsoft--Phi-3.5-mini-instruct/snapshots/af0dfb8029e8a74545d0736d30cb6b58d2f0f3f0/config.json
Model config Phi3Config {
  "_name_or_path": "Phi-3.5-mini-instruct",
  "architectures": [
    "Phi3ForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "auto_map": {
    "AutoConfig": "microsoft/Phi-3.5-mini-instruct--configuration_phi3.Phi3Config",
    "AutoModelForCausalLM": "microsoft/Phi-3.5-mini-instruct--modeling_phi3.Phi3ForCausalLM"
  },
  "bos_token_id": 1,
  "embd_pdrop": 0.0,
  "eos_token_id": 32000,
  "hidden_act": "silu",
  "hidden_size": 3072,
  "initializer_range": 0.02,
  "intermediate_size": 8192,
  "max_position_embeddings": 131072,
  "model_type": "phi3",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 32,
  "original_max_position_embeddings": 4096,
  "pad_token_id": 3200

TrainOutput(global_step=150, training_loss=0.9603229363759359, metrics={'train_runtime': 830.1427, 'train_samples_per_second': 0.72, 'train_steps_per_second': 0.181, 'total_flos': 4073714419630080.0, 'train_loss': 0.9603229363759359})

In [38]:
# Start training
trainer.train() 

The following columns in the training set don't have a corresponding argument in `PeftModelForCausalLM.forward` and have been ignored: text. If text are not expected by `PeftModelForCausalLM.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 299
  Num Epochs = 2
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 150
  Number of trainable parameters = 8,912,896


Step,Training Loss
20,0.818200
40,0.749000
60,0.725700
80,0.737100
100,0.701600
120,0.711900
140,0.695300


Saving model checkpoint to ./checkpoint_dir/checkpoint-100
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--microsoft--Phi-3.5-mini-instruct/snapshots/af0dfb8029e8a74545d0736d30cb6b58d2f0f3f0/config.json
Model config Phi3Config {
  "_name_or_path": "Phi-3.5-mini-instruct",
  "architectures": [
    "Phi3ForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "auto_map": {
    "AutoConfig": "microsoft/Phi-3.5-mini-instruct--configuration_phi3.Phi3Config",
    "AutoModelForCausalLM": "microsoft/Phi-3.5-mini-instruct--modeling_phi3.Phi3ForCausalLM"
  },
  "bos_token_id": 1,
  "embd_pdrop": 0.0,
  "eos_token_id": 32000,
  "hidden_act": "silu",
  "hidden_size": 3072,
  "initializer_range": 0.02,
  "intermediate_size": 8192,
  "max_position_embeddings": 131072,
  "model_type": "phi3",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 32,
  "original_max_position_embeddings": 4096,
  "pad_token_id": 3200

TrainOutput(global_step=150, training_loss=0.7322171370188395, metrics={'train_runtime': 830.08, 'train_samples_per_second': 0.72, 'train_steps_per_second': 0.181, 'total_flos': 4073714419630080.0, 'train_loss': 0.7322171370188395})

In [48]:
# Start training
trainer.train() 

The following columns in the training set don't have a corresponding argument in `PeftModelForCausalLM.forward` and have been ignored: text. If text are not expected by `PeftModelForCausalLM.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 299
  Num Epochs = 2
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 150
  Number of trainable parameters = 8,912,896


Step,Training Loss
20,0.736100
40,0.683200
60,0.667700
80,0.687900
100,0.653200
120,0.665500
140,0.648100


Saving model checkpoint to ./checkpoint_dir/checkpoint-100
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--microsoft--Phi-3.5-mini-instruct/snapshots/af0dfb8029e8a74545d0736d30cb6b58d2f0f3f0/config.json
Model config Phi3Config {
  "_name_or_path": "Phi-3.5-mini-instruct",
  "architectures": [
    "Phi3ForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "auto_map": {
    "AutoConfig": "microsoft/Phi-3.5-mini-instruct--configuration_phi3.Phi3Config",
    "AutoModelForCausalLM": "microsoft/Phi-3.5-mini-instruct--modeling_phi3.Phi3ForCausalLM"
  },
  "bos_token_id": 1,
  "embd_pdrop": 0.0,
  "eos_token_id": 32000,
  "hidden_act": "silu",
  "hidden_size": 3072,
  "initializer_range": 0.02,
  "intermediate_size": 8192,
  "max_position_embeddings": 131072,
  "model_type": "phi3",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 32,
  "original_max_position_embeddings": 4096,
  "pad_token_id": 3200

TrainOutput(global_step=150, training_loss=0.6758983707427979, metrics={'train_runtime': 830.3213, 'train_samples_per_second': 0.72, 'train_steps_per_second': 0.181, 'total_flos': 4073714419630080.0, 'train_loss': 0.6758983707427979})

In [49]:
# Start training
trainer.train() 

The following columns in the training set don't have a corresponding argument in `PeftModelForCausalLM.forward` and have been ignored: text. If text are not expected by `PeftModelForCausalLM.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 299
  Num Epochs = 2
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 150
  Number of trainable parameters = 8,912,896


Step,Training Loss
20,0.689700
40,0.636300
60,0.624000
80,0.646900
100,0.613800
120,0.625800
140,0.608400


Saving model checkpoint to ./checkpoint_dir/checkpoint-100
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--microsoft--Phi-3.5-mini-instruct/snapshots/af0dfb8029e8a74545d0736d30cb6b58d2f0f3f0/config.json
Model config Phi3Config {
  "_name_or_path": "Phi-3.5-mini-instruct",
  "architectures": [
    "Phi3ForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "auto_map": {
    "AutoConfig": "microsoft/Phi-3.5-mini-instruct--configuration_phi3.Phi3Config",
    "AutoModelForCausalLM": "microsoft/Phi-3.5-mini-instruct--modeling_phi3.Phi3ForCausalLM"
  },
  "bos_token_id": 1,
  "embd_pdrop": 0.0,
  "eos_token_id": 32000,
  "hidden_act": "silu",
  "hidden_size": 3072,
  "initializer_range": 0.02,
  "intermediate_size": 8192,
  "max_position_embeddings": 131072,
  "model_type": "phi3",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 32,
  "original_max_position_embeddings": 4096,
  "pad_token_id": 3200

TrainOutput(global_step=150, training_loss=0.6336089420318604, metrics={'train_runtime': 830.3564, 'train_samples_per_second': 0.72, 'train_steps_per_second': 0.181, 'total_flos': 4073714419630080.0, 'train_loss': 0.6336089420318604})

In [50]:
# Start training
trainer.train() 

The following columns in the training set don't have a corresponding argument in `PeftModelForCausalLM.forward` and have been ignored: text. If text are not expected by `PeftModelForCausalLM.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 299
  Num Epochs = 2
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 150
  Number of trainable parameters = 8,912,896


Step,Training Loss
20,0.649800
40,0.596800
60,0.586200
80,0.611100
100,0.578900
120,0.590500
140,0.573400


Saving model checkpoint to ./checkpoint_dir/checkpoint-100
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--microsoft--Phi-3.5-mini-instruct/snapshots/af0dfb8029e8a74545d0736d30cb6b58d2f0f3f0/config.json
Model config Phi3Config {
  "_name_or_path": "Phi-3.5-mini-instruct",
  "architectures": [
    "Phi3ForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "auto_map": {
    "AutoConfig": "microsoft/Phi-3.5-mini-instruct--configuration_phi3.Phi3Config",
    "AutoModelForCausalLM": "microsoft/Phi-3.5-mini-instruct--modeling_phi3.Phi3ForCausalLM"
  },
  "bos_token_id": 1,
  "embd_pdrop": 0.0,
  "eos_token_id": 32000,
  "hidden_act": "silu",
  "hidden_size": 3072,
  "initializer_range": 0.02,
  "intermediate_size": 8192,
  "max_position_embeddings": 131072,
  "model_type": "phi3",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 32,
  "original_max_position_embeddings": 4096,
  "pad_token_id": 3200

TrainOutput(global_step=150, training_loss=0.5965744558970133, metrics={'train_runtime': 830.5615, 'train_samples_per_second': 0.72, 'train_steps_per_second': 0.181, 'total_flos': 4073714419630080.0, 'train_loss': 0.5965744558970133})

In [51]:
# Start training
trainer.train() 

The following columns in the training set don't have a corresponding argument in `PeftModelForCausalLM.forward` and have been ignored: text. If text are not expected by `PeftModelForCausalLM.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 299
  Num Epochs = 2
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 150
  Number of trainable parameters = 8,912,896


Step,Training Loss
20,0.615000
40,0.561800
60,0.553800
80,0.578400
100,0.549000
120,0.559900
140,0.542200


Saving model checkpoint to ./checkpoint_dir/checkpoint-100
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--microsoft--Phi-3.5-mini-instruct/snapshots/af0dfb8029e8a74545d0736d30cb6b58d2f0f3f0/config.json
Model config Phi3Config {
  "_name_or_path": "Phi-3.5-mini-instruct",
  "architectures": [
    "Phi3ForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "auto_map": {
    "AutoConfig": "microsoft/Phi-3.5-mini-instruct--configuration_phi3.Phi3Config",
    "AutoModelForCausalLM": "microsoft/Phi-3.5-mini-instruct--modeling_phi3.Phi3ForCausalLM"
  },
  "bos_token_id": 1,
  "embd_pdrop": 0.0,
  "eos_token_id": 32000,
  "hidden_act": "silu",
  "hidden_size": 3072,
  "initializer_range": 0.02,
  "intermediate_size": 8192,
  "max_position_embeddings": 131072,
  "model_type": "phi3",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 32,
  "original_max_position_embeddings": 4096,
  "pad_token_id": 3200

TrainOutput(global_step=150, training_loss=0.5641629791259766, metrics={'train_runtime': 830.3607, 'train_samples_per_second': 0.72, 'train_steps_per_second': 0.181, 'total_flos': 4073714419630080.0, 'train_loss': 0.5641629791259766})

In [17]:
import torch
torch.cuda.empty_cache()  # Clears cached memory

In [ ]:
# Save the fine-tuned model

trainer.save_model("./checkpoint_dir")

In [ ]:
import shutil
import os

# Specify the directory path you want to compress
dir_path = "./checkpoint_dir"  # Replace with your folder path

# Compress the directory into a zip file
shutil.make_archive("/kaggle/working/Sra1emotional_assistant", "zip", dir_path)

# Check if the zip file was created successfully
if os.path.exists("/kaggle/working/Sra1emotional_assistant.zip"):
    print("Zip file created: Sra1 emotional chatbot.zip")
else:
    print("Failed to create zip file.")
